In [1]:
import gensim
import string
import nltk
import sparknlp
import numpy as np

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /common/home/ac1771/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /common/home/ac1771/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df_raw = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')

In [3]:
df_raw = df_raw.na.drop(subset=["selected_text"])

In [6]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

# Remove punctuation, stop words, and lower case the letters
def remove_stopwords(line):
    line = line[:line.index('http')] if 'http' in line else line
    line = re.sub(r'[^a-zA-Z ]+', '', line)
    tokens = word_tokenize(line)
    remove_stopwords = [lemmatize_stemming(t.lower()) for t in tokens if not t in STOPWORDS and not t in string.punctuation and len(t) > 2]
    return ",".join(remove_stopwords)

remove_stopwords_udf = F.udf(lambda z: remove_stopwords(z))
processed_df = df_raw.withColumn("cleaned_text", remove_stopwords_udf(F.col("selected_text")))

In [11]:
processed_df.tail(5)

[Row(selected_text='Florida Ammo Selling Out On Heels of Stay-At-Home Order', cleaned_text='florida,ammo,sell,out,heel,stayathome,order'),
 Row(selected_text='Disney Forcing Annual Pass Holders to Continue Paying, but Adding Park Days', cleaned_text='disney,force,annual,pass,holders,continue,pay,add,park,days'),
 Row(selected_text='Nick Cannon Pimps Out His Impala with Custom Nipsey Hussle Tribute', cleaned_text='nick,cannon,pimp,out,his,impala,custom,nipsey,hussle,tribute'),
 Row(selected_text='Pete Buttigieg Says Governors Showing More Leadership Than Trump', cleaned_text='pete,buttigieg,say,governors,show,more,leadership,than,trump'),
 Row(selected_text='Ruth Bader Ginsburg Still Working Out with Trainer Despite Coronavirus', cleaned_text='ruth,bader,ginsburg,still,work,out,trainer,despite,coronavirus')]

In [24]:
processed_df.printSchema()

root
 |-- selected_text: string (nullable = true)
 |-- cleaned_text: string (nullable = true)



In [25]:
processed_df.write.mode("Overwrite").json('/common/users/shared/cs543_fall22_group3/combined/combined_processed')

In [19]:
selected = processed_df.take(15100)
cleaned = processed_df.take(15100)

In [23]:
for i in range(15020, 150100):
    print('Raw text: {}'.format(selected[i].selected_text))
    print('Cleaned text: {}'.format(cleaned[i].cleaned_text))
    print()

Raw text: CBI books agro firm for siphoning off Rs 3.5 crore
Cleaned text: cbi,book,agro,firm,siphon,crore

Raw text: Pakistan violates truce along IB in J&K
Cleaned text: pakistan,violate,truce

Raw text: Call centre to assist in post-Covid care in Jaisalmer
Cleaned text: call,centre,assist,postcovid,care,jaisalmer

Raw text: mumbai cet for degree colleges may ensure fair merit list says principals
Cleaned text: mumbai,cet,degree,colleges,ensure,fair,merit,list,say,principals

Raw text: Covid death: RIL off-rolls to benefit too
Cleaned text: covid,death,ril,offrolls,benefit

Raw text: Post-Covid ailments increasing among kids
Cleaned text: postcovid,ailments,increase,kid

Raw text: Exams Cancelled; 12th Students And Schools Await Result Criteria
Cleaned text: exams,cancel,students,and,school,await,result,criteria

Raw text: Kerala CM Pinarayi Vijayan: Only Centre can ensure affordable petro products
Cleaned text: kerala,pinarayi,vijayan,only,centre,ensure,affordable,petro,products

Ra

IndexError: list index out of range